In [34]:
import pandas as pd
import numpy as np




In [35]:
data = pd.read_csv('Dataset_1.csv')
data

,order_id,customer_id,customer_age,order_amount,order_date,payment_mode,city,delivery_days,is_delivered
0,ORD1000,NaN,NaN,250.5,2023-01-15,Debit Card,NaN,5,True
1,ORD1001,NaN,18.0,1200,2023/03/10,Debit Card,NaN,5,yes
2,ORD1002,CUST2,35.0,NaN,NaN,NaN,Mumbai,7,yes
3,ORD1003,user_3,35.0,1200,2023-01-15,Cash,NaN,three,False
4,ORD1004,user_4,-3.0,3500.75,2023-01-15,CARD,NaN,three,yes
...,...,...,...,...,...,...,...,...,...
995,NaN,user_995,-3.0,3500.75,15/02/2023,upi,Mumbai,1,True
996,NaN,CUST996,18.0,NaN,2023-01-15,Credit Card,Hyderabad,2,yes
997,NaN,user_997,50.0,NaN,2023/03/10,Cash,Delhi,1,NaN
998,ORD1998,NaN,65.0,1200,2023-01-15,NaN,Mumbai,2,False


## 1st-


In [40]:
mask = pd.to_numeric(data['order_amount'], errors='coerce').isna() & data['is_delivered'].astype(str).str.lower().isin(["true","yes"])
data[mask]

,order_id,customer_id,customer_age,order_amount,order_date,payment_mode,city,delivery_days,is_delivered
2,ORD1002,CUST2,35.0,NaN,NaN,NaN,Mumbai,7,yes
14,ORD1014,CUST14,25.0,NaN,2023/03/10,upi,DELHI,5,yes
17,NaN,NaN,-3.0,NaN,15/02/2023,upi,Hyderabad,NaN,True
24,ORD1024,CUST24,NaN,₹999,invalid,Credit Card,DELHI,1,yes
27,NaN,NaN,25.0,₹999,2023-01-15,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...
985,ORD1985,CUST985,35.0,₹999,15/02/2023,Debit Card,Mumbai,5,True
987,NaN,CUST987,50.0,NaN,2023/03/10,NaN,Bangalore,NaN,yes
990,NaN,NaN,25.0,₹999,15/02/2023,upi,Hyderabad,three,True
996,NaN,CUST996,18.0,NaN,2023-01-15,Credit Card,Hyderabad,2,yes


## 2nd- Convert order_date to datetime and find orders where delivery_days is negative after valid date conversion

In [14]:
data['order_date'] = pd.to_datetime(data['order_date'], errors='coerce', dayfirst=True)

data['delivery_days'] = pd.to_numeric(data['delivery_days'], errors='coerce')
ans = data['delivery_days'] < 0 & data['order_date'].notna()
data[ans]

,order_id,customer_id,customer_age,order_amount,order_date,payment_mode,city,delivery_days,is_delivered,delivery_days_numeric
9,NaN,user_9,NaN,NaN,NaT,UPI,Delhi,-1.0,no,-1.0
11,NaN,CUST11,18.0,3500.75,NaT,Cash,Mumbai,-1.0,False,-1.0
35,NaN,user_35,50.0,499.99,NaT,Cash,DELHI,-1.0,yes,-1.0
47,NaN,user_47,25.0,NaN,NaT,Cash,Mumbai,-1.0,NaN,-1.0
49,ORD1049,CUST49,120.0,3500.75,NaT,NaN,Hyderabad,-1.0,False,-1.0
...,...,...,...,...,...,...,...,...,...,...
936,ORD1936,user_936,NaN,NaN,NaT,UPI,Bangalore,-1.0,no,-1.0
937,ORD1937,CUST937,-3.0,NaN,2023-01-15,Debit Card,mumbai,-1.0,yes,-1.0
938,ORD1938,user_938,65.0,250.5,NaT,upi,mumbai,-1.0,NaN,-1.0
945,ORD1945,NaN,NaN,250.5,2023-01-15,NaN,mumbai,-1.0,no,-1.0


## 3rd-Standardize payment_mode and calculate the percentage share of each per city

In [35]:
# Standardize payment_mode and calculate the percentage share of each per city

# Create a mapping for payment_mode codes to readable names if needed
payment_mode_map = {
    1: 'Cash',
    2: 'Card',
    3: 'UPI',
    4: 'Debit Card',
    5: 'Net Banking'
}

# Replace payment_mode codes with readable names if not already done
data['payment_mode_std'] = data['payment_mode'].map(payment_mode_map).fillna(data['payment_mode'])

# Drop rows where city or payment_mode is missing for accurate calculation
city_payment = data.dropna(subset=['city', 'payment_mode_std'])

# Calculate counts per city and payment_mode
counts = city_payment.groupby(['city', 'payment_mode_std']).size().reset_index(name='count')

# Calculate total orders per city
total_per_city = counts.groupby('city')['count'].transform('sum')

# Calculate percentage share
counts['percentage'] = (counts['count'] / total_per_city * 100).round(2)

counts

,city,payment_mode_std,count,percentage
0,Bangalore,Cash,151,100.0
1,DELHI,Cash,141,100.0
2,Delhi,Cash,161,100.0
3,Hyderabad,Cash,141,100.0
4,Mumbai,Cash,145,100.0
5,mumbai,Cash,119,100.0


## 4- Detect customers whose customer_age is invalid but have order amounts above the dataset median

In [30]:
data

,order_id,customer_id,customer_age,order_amount,order_date,payment_mode,city,delivery_days,is_delivered,delivery_days_numeric
0,ORD1000,NaN,NaN,250.5,2023-01-15,1,NaN,5.0,True,5.0
1,ORD1001,NaN,18.0,1200,NaT,1,NaN,5.0,yes,5.0
2,ORD1002,CUST2,35.0,NaN,NaT,1,Mumbai,7.0,yes,7.0
3,ORD1003,user_3,35.0,1200,2023-01-15,1,NaN,NaN,False,NaN
4,ORD1004,user_4,-3.0,3500.75,2023-01-15,1,NaN,NaN,yes,NaN
...,...,...,...,...,...,...,...,...,...,...
995,NaN,user_995,-3.0,3500.75,NaT,1,Mumbai,1.0,True,1.0
996,NaN,CUST996,18.0,NaN,2023-01-15,1,Hyderabad,2.0,yes,2.0
997,NaN,user_997,50.0,NaN,NaT,1,Delhi,1.0,NaN,1.0
998,ORD1998,NaN,65.0,1200,2023-01-15,1,Mumbai,2.0,False,2.0


In [42]:

invalid = (data['customer_age'] < 0) | (data['customer_age'] > 100)

order_amount_numeric = pd.to_numeric(data['order_amount'], errors='coerce')

median = order_amount_numeric.median()

result = data[invalid & (order_amount_numeric > median)]
result





,order_id,customer_id,customer_age,order_amount,order_date,payment_mode,city,delivery_days,is_delivered,delivery_days_numeric,payment_mode_std
4,ORD1004,user_4,-3,3500.75,2023-01-15,1,NaN,NaN,yes,NaN,Cash
34,ORD1034,NaN,120,3500.75,NaT,1,Hyderabad,7.0,True,7.0,Cash
49,ORD1049,CUST49,120,3500.75,NaT,1,Hyderabad,-1.0,False,-1.0,Cash
53,NaN,user_53,120,3500.75,NaT,1,Hyderabad,NaN,NaN,NaN,Cash
59,NaN,NaN,-3,3500.75,NaT,1,Bangalore,NaN,False,NaN,Cash
...,...,...,...,...,...,...,...,...,...,...,...
954,NaN,user_954,120,1200.00,2023-01-15,1,mumbai,1.0,yes,1.0,Cash
971,ORD1971,NaN,120,1200.00,NaT,1,Delhi,NaN,NaN,NaN,Cash
988,NaN,NaN,120,3500.75,NaT,1,Bangalore,2.0,yes,2.0,Cash
991,NaN,NaN,-3,3500.75,NaT,1,Hyderabad,1.0,yes,1.0,Cash


## 5- find duplicate customer_ids where the same customer placed orders in multiple cities.

In [52]:
unique_city = data.groupby('customer_id')['city'].nunique()

customer_order = unique_city[unique_city >= 1].reset_index()
customer_order

,customer_id,city
0,CUST108,1
1,CUST11,1
2,CUST111,1
3,CUST117,1
4,CUST119,1
...,...,...
408,user_959,1
409,user_97,1
410,user_977,1
411,user_995,1


## 6- Create a flag for orders where delivery_days is missing but is_delivered is True

In [60]:
#nan or true -> true
#nan or false -> false
#true or true -> false
#false or false -> false

data['order_placed_status'] = data['delivery_days'].isna() & (data['is_delivered'].astype(str).str.lower().isin(["true","yes"]))
data['order_placed_status'].value_counts()

order_placed_status
False    885
True     115
Name: count, dtype: int64

## 7- After cleaning, compute the average order value per payment mode, excluding invalid amounts

In [66]:

# valid_orders = data[(data['order_amount'].notna()) & (data['order_amount'] > 0)]


average_order_value = valid_orders.groupby('payment_mode_std')['order_amount'].mean()

res = average_order_value.reset_index(name="Avg_Order_Value")
res





,payment_mode_std,Avg_Order_Value
0,Cash,1429.183717


## 8- Identify cities where more than 30% of orders have missing order_date

In [77]:
city_missing = (
    data.assign(missing=data['order_date'].isna())
        .groupby('city')['missing']
        .mean()
        .mul(100)
)

city_missing[city_missing > 30]


city
Bangalore    80.794702
DELHI        78.723404
Delhi        81.987578
Hyderabad    84.397163
Mumbai       75.862069
mumbai       72.268908
Name: missing, dtype: float64